# Snakes Game Using Pygame And doing it with DL

In [1]:
import pygame
import numpy as np
import time
import random


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


# Rules Of Games
1. If Collided with Apple, Score incremented by 1
2. If COllided with boundaries or with itself, return 1 that is out
3. For LEFT -> button_direction = 0
       RIGHT -> button_direction = 1 
       DOWN ->button_direction = 2
       UP -> button_direction = 3

In [11]:
def collision_with_apple(apple_position, score):
    apple_position = [random.randrange(1,50)*10,random.randrange(1,50)*10]
    score += 1
    return apple_position, score

def collision_with_boundaries(snake_head):
    if snake_head[0]>=500 or snake_head[0]<0 or snake_head[1]>=500 or snake_head[1]<0 :
        return 1
    else:
        return 0

def collision_with_self(snake_position):
    snake_head = snake_position[0]
    if snake_head in snake_position[1:]:
        return 1
    else:
        return 0

In [12]:
def is_direction_blocked(snake_position, current_direction_vector):
    next_step = snake_position[0]+ current_direction_vector
    snake_head = snake_position[0]
    if collision_with_boundaries(snake_head) == 1 or collision_with_self(snake_position) == 1:
        return 1
    else:
        return 0

In [13]:
def generate_snake(snake_head, snake_position, apple_position, button_direction, score):

    if button_direction == 1:
        snake_head[0] += 10
    elif button_direction == 0:
        snake_head[0] -= 10
    elif button_direction == 2:
        snake_head[1] += 10
    elif button_direction == 3:
        snake_head[1] -= 10
    else:
        pass
        
    if snake_head == apple_position:
        apple_position, score = collision_with_apple(apple_position, score)
        snake_position.insert(0,list(snake_head))

    else:
        snake_position.insert(0,list(snake_head))
        snake_position.pop()
        
    return snake_position, apple_position, score

In [14]:
def display_snake(snake_position):
    for position in snake_position:
        pygame.draw.rect(display,red,pygame.Rect(position[0],position[1],10,10))

def display_apple(display,apple_position, apple):
    display.blit(apple,(apple_position[0], apple_position[1]))
    

In [18]:
def play_game(snake_head, snake_position, apple_position, button_direction, apple, score):
    crashed = False
    prev_button_direction = 1
    button_direction = 1
    current_direction_vector = np.array(snake_position[0])-np.array(snake_position[1])

    while crashed is not True:
        for event in pygame.event.get():

            if event.type == pygame.QUIT:
                crashed = True
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT and prev_button_direction != 1:
                    button_direction = 0
                elif event.key == pygame.K_RIGHT and prev_button_direction != 0:
                    button_direction = 1
                elif event.key == pygame.K_UP and prev_button_direction != 2:
                    button_direction = 3
                elif event.key == pygame.K_DOWN and prev_button_direction != 3:
                    button_direction = 2
                else:
                    button_direction = button_direction
        
        display.fill(window_color)
        display_apple(display,apple_position,apple)
        display_snake(snake_position)

        snake_position, apple_position, score = generate_snake(snake_head, snake_position, apple_position, button_direction, score)
        pygame.display.set_caption("Snake Game"+"  "+"SCORE: "+str(score))
        pygame.display.update()
        prev_button_direction = button_direction
        if is_direction_blocked(snake_position, current_direction_vector) == 1:
            crashed = True

        clock.tick(2)
    return score


In [ ]:
def display_final_score(display_text, final_score):
    largeText = pygame.font.Font('freesansbold.ttf',35)
    TextSurf = largeText.render(display_text, True, black)
    TextRect = TextSurf.get_rect()
    TextRect.center = ((display_width/2),(display_height/2))
    display.blit(TextSurf, TextRect)
    pygame.display.update()
    time.sleep(2)

In [ ]:
if __name__ == "__main__":
    
###### initialize required parameters ########  
    display_width = 500
    display_height = 500
    green = (0,255,0)
    red = (255,0,0)
    black = (0,0,0)
    window_color = (200,200,200)
    apple_image = pygame.image.load('apple.jpg')
    clock=pygame.time.Clock() 
    
    snake_head = [250,250]
    snake_position = [[250,250],[240,250],[230,250]]
    apple_position = [random.randrange(1,50)*10,random.randrange(1,50)*10]
    score = 0
    
    pygame.init() #initialize pygame modules    

    #### display game window #####

    display = pygame.display.set_mode((display_width,display_height))
    display.fill(window_color)
    pygame.display.update()
    
    final_score = play_game(snake_head, snake_position, apple_position, 1, apple_image, score)
    display = pygame.display.set_mode((display_width,display_height))
    display.fill(window_color)
    pygame.display.update()

    display_text = 'Your Score is: ' + str(final_score)
    display_final_score(display_text, final_score)

    pygame.quit()



### Now Let's Train the position and let the computer play the game and achieve good scores.

In [ ]:
from game import *
from training_data import generate_training_data

from keras.models import Sequential
from keras.layers import Dense

display_width = 500
display_height = 500
green = (0,255,0)
red = (255,0,0)
black = (0,0,0)
white = (255,255,255)

pygame.init()
display=pygame.display.set_mode((display_width,display_height))
clock=pygame.time.Clock()

'''
LEFT -> button_direction = 0
RIGHT -> button_direction = 1
DOWN ->button_direction = 2
UP -> button_direction = 3
'''
training_data_x, training_data_y = generate_training_data(display,clock)


model = Sequential()
model.add(Dense(units=9,input_dim=7))

model.add(Dense(units=15, activation='relu'))
model.add(Dense(output_dim=3,  activation = 'softmax'))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.fit((np.array(training_data_x).reshape(-1,7)),( np.array(training_data_y).reshape(-1,3)), batch_size = 256,epochs= 3)

model.save_weights('model.h5')
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

  5%|▌         | 53/1000 [03:41<1:37:13,  6.16s/it]

####                                                                                                                                                     
                                                                            Bertelsmann Udacity Challenge AI
                                                                            By : Aayushi Jain
                                                                            Slack handle : @Aayushi Jain